#### This Version Creates the Graph in Neo4J then runs the Queries.

In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [5]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_2_4')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351/import'

# Specify the filenames to move
filenames = ['spectra.csv', 'peakrange.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [6]:
# Define the working directory
working_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351/bin'

# Construct the command
# command = [
#     './bin/neo4j-admin', 'database', 'import', 'full',
#     '--nodes=import/nodes.csv',
#     '--relationships=import/relationships.csv', 'neo4j'
# ]

command = 'neo4j-admin database import full --nodes=import/spectra.csv --nodes=import/peakrange.csv --relationships=import/relationships.csv --overwrite-destination neo4j'

# Execute the command


result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)
#result = subprocess.run(command, cwd=working_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
WARN: file group with header file C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\spectra.csv specifies no node labels, which could be a mistake
WARN: file group with header file C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\peakrange.csv specifies no node labels, which could be a mistake
Neo4j version: 5.12.0
Importing the contents of these files into C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\data\databases\neo4j:
Nodes:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\spectra.csv
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\peakrange.csv

Relationships:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\relationships.csv


Available resources:
  Total machine memory: 31.86GiB
 

Run and connect to the Neo4j Database

In [7]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [5]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', ''],
      {
        LINK: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [6]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [7]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [8]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [9]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [10]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [11]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [12]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [13]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [14]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'Weight',
         randomSeed:1234,
         embeddingDimension: 128
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [15]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [16]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'Louvain'])

In [17]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['name'], how='left')

In [18]:
df = merged_df.rename(columns={'name' : 'SpecID'})
df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,210407-1-22,1.151713,2301.560127,0.022129,0.238165,2,1122
1,210407-1-29,1.142775,2280.548482,0.021927,0.237365,2,1122
2,210519-1-20,1.141597,2277.600657,0.021898,0.237256,2,1122
3,210511-1-49,1.138794,2270.600562,0.021831,0.236992,2,1122
4,210504-3-36,1.138285,2272.546214,0.021850,0.237036,1593,404
...,...,...,...,...,...,...,...
3040,210303-1-48,0.415566,602.611228,0.005794,0.173155,2,1122
3041,210324-1-10,0.412717,596.614308,0.005736,0.172919,421,477
3042,210211-1-08,0.382157,523.342455,0.005032,0.170141,421,477
3043,210303-2-26,0.380200,525.173999,0.005049,0.170157,2,1122


In [19]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,-0.065681,0.088482,0.332627,0.081993,0.278461,0.024700,-0.184946,-0.065128,0.123959,...,-0.092620,0.111903,-0.079377,0.194498,0.298167,-0.099803,0.083769,-0.264178,-0.020173,-0.097964
1,201210-1-01,-0.065622,0.086918,0.329847,0.087437,0.280502,0.025002,-0.181201,-0.066186,0.123731,...,-0.093857,0.113799,-0.078777,0.192893,0.296552,-0.097552,0.084439,-0.269068,-0.020639,-0.099946
2,201210-1-02,-0.064439,0.089211,0.333425,0.082610,0.279080,0.026678,-0.184957,-0.065443,0.123558,...,-0.092693,0.111522,-0.079684,0.193833,0.298340,-0.099026,0.083939,-0.264206,-0.019721,-0.098134
3,201210-1-03,-0.066912,0.088093,0.332399,0.082638,0.278347,0.025275,-0.184075,-0.064797,0.124690,...,-0.092978,0.113499,-0.079315,0.195707,0.296758,-0.100171,0.084045,-0.264537,-0.019515,-0.098897
4,201210-1-04,-0.065925,0.087227,0.330243,0.087199,0.280096,0.025165,-0.181506,-0.066416,0.123983,...,-0.093782,0.113566,-0.078899,0.193446,0.296808,-0.097671,0.084302,-0.268745,-0.020502,-0.100226


Delete the projection

In [20]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [21]:
close_driver()

In [22]:
status_df = pd.read_csv('../../data/graph_2_3/nodes.csv')

In [23]:
status_df

,Unnamed: 0,SpecID:ID,Status:LABEL,SurID
0,0,201210-1-00,Normal,201210-1
1,1,201210-1-01,Normal,201210-1
2,2,201210-1-02,Normal,201210-1
3,3,201210-1-03,Normal,201210-1
4,4,201210-1-04,Normal,201210-1
...,...,...,...,...
3040,3040,210526-3-45,Hyperglycemia,210526-3
3041,3041,210526-3-46,Hyperglycemia,210526-3
3042,3042,210526-3-47,Hyperglycemia,210526-3
3043,3043,210526-3-48,Hyperglycemia,210526-3


In [24]:
status_df.drop(columns=['Unnamed: 0'], inplace=True)

In [25]:
graph_df = pd.merge(df, status_df, left_on="SpecID", right_on="SpecID:ID", how="inner")
graph_df.drop(columns=["SpecID:ID"], inplace=True)

In [26]:
fast_graph_df = pd.merge(fastRP_df, status_df, left_on="SpecID", right_on="SpecID:ID", how="inner")
fast_graph_df.drop(columns=["SpecID:ID"], inplace=True)

In [27]:
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [28]:
fast_graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [29]:
fast_graph_df

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,Status,SurID
0,201210-1-00,-0.065681,0.088482,0.332627,0.081993,0.278461,0.024700,-0.184946,-0.065128,0.123959,...,-0.079377,0.194498,0.298167,-0.099803,0.083769,-0.264178,-0.020173,-0.097964,Normal,201210-1
1,201210-1-01,-0.065622,0.086918,0.329847,0.087437,0.280502,0.025002,-0.181201,-0.066186,0.123731,...,-0.078777,0.192893,0.296552,-0.097552,0.084439,-0.269068,-0.020639,-0.099946,Normal,201210-1
2,201210-1-02,-0.064439,0.089211,0.333425,0.082610,0.279080,0.026678,-0.184957,-0.065443,0.123558,...,-0.079684,0.193833,0.298340,-0.099026,0.083939,-0.264206,-0.019721,-0.098134,Normal,201210-1
3,201210-1-03,-0.066912,0.088093,0.332399,0.082638,0.278347,0.025275,-0.184075,-0.064797,0.124690,...,-0.079315,0.195707,0.296758,-0.100171,0.084045,-0.264537,-0.019515,-0.098897,Normal,201210-1
4,201210-1-04,-0.065925,0.087227,0.330243,0.087199,0.280096,0.025165,-0.181506,-0.066416,0.123983,...,-0.078899,0.193446,0.296808,-0.097671,0.084302,-0.268745,-0.020502,-0.100226,Normal,201210-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,210526-3-45,-0.066367,0.086022,0.329202,0.087400,0.281091,0.025478,-0.180275,-0.066638,0.123629,...,-0.079942,0.191062,0.296821,-0.097246,0.083716,-0.268692,-0.020020,-0.098902,Hyperglycemia,210526-3
3041,210526-3-46,-0.065594,0.086803,0.330848,0.087468,0.280446,0.026551,-0.181218,-0.065023,0.123401,...,-0.078844,0.193598,0.295869,-0.097495,0.085071,-0.269066,-0.020303,-0.099775,Hyperglycemia,210526-3
3042,210526-3-47,-0.066576,0.086610,0.330607,0.087472,0.280890,0.026827,-0.180395,-0.066008,0.123999,...,-0.080093,0.191789,0.296591,-0.097119,0.084318,-0.268481,-0.019801,-0.099437,Hyperglycemia,210526-3
3043,210526-3-48,-0.065769,0.086933,0.331084,0.086751,0.280112,0.026233,-0.181737,-0.064930,0.123564,...,-0.078846,0.193995,0.296054,-0.097745,0.084976,-0.268520,-0.020271,-0.099528,Hyperglycemia,210526-3


In [30]:
graph_df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,Status,SurID
0,210407-1-22,1.151713,2301.560127,0.022129,0.238165,2,1122,Hypoglycemia,210407-1
1,210407-1-29,1.142775,2280.548482,0.021927,0.237365,2,1122,Hypoglycemia,210407-1
2,210519-1-20,1.141597,2277.600657,0.021898,0.237256,2,1122,Hyperglycemia,210519-1
3,210511-1-49,1.138794,2270.600562,0.021831,0.236992,2,1122,Hyperglycemia,210511-1
4,210504-3-36,1.138285,2272.546214,0.021850,0.237036,1593,404,Hypoglycemia,210504-3
...,...,...,...,...,...,...,...,...,...
3040,210303-1-48,0.415566,602.611228,0.005794,0.173155,2,1122,Normal,210303-1
3041,210324-1-10,0.412717,596.614308,0.005736,0.172919,421,477,Normal,210324-1
3042,210211-1-08,0.382157,523.342455,0.005032,0.170141,421,477,Hyperglycemia,210211-1
3043,210303-2-26,0.380200,525.173999,0.005049,0.170157,2,1122,Normal,210303-2


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import numpy as np

def evaluate_extra_trees(df):

    # Set the Surfaces as groups
    groups = graph_df['SurID']
    X = graph_df.drop(['Status', 'SurID', 'SpecID'], axis=1)
    y = graph_df['Status']

    # Creating the Extra Trees classifier
    et = ExtraTreesClassifier(random_state=1234)
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)

    scores = []
    for train_index, test_index in cv.split(X, y, groups):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the Extra Trees classifier
        et.fit(X_train, y_train)
        predictions = et.predict(X_test)
        
        # Evaluate the model
        score = accuracy_score(y_test, predictions)
        scores.append(score)
    
    # Displaying the results
    print(f'{et.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

In [32]:
evaluate_extra_trees(graph_df)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3436 +/- 0.0308


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import numpy as np

# Splitting the dataframe into features (X) and target variable (y)
X = graph_df.drop(['Status', 'SpecID', 'SurID'], axis=1)
y = graph_df['Status']

# Creating the classifiers
rf = RandomForestClassifier(random_state=1234)
et = ExtraTreesClassifier(random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.3678 +/- 0.0236
ExtraTreesClassifier Cross-Validation Accuracy: 0.3714 +/- 0.0160


In [34]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

In [35]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [36]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [37]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.38752052545155996


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.34      0.40      0.37       178
 Hypoglycemia       0.40      0.36      0.38       204
       Normal       0.42      0.40      0.41       227

     accuracy                           0.39       609
    macro avg       0.39      0.39      0.39       609
 weighted avg       0.39      0.39      0.39       609


Confusion Matrix:
[[72 46 60]
 [66 74 64]
 [74 63 90]]


In [38]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.39080459770114945


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.34      0.40      0.37       178
 Hypoglycemia       0.39      0.35      0.37       204
       Normal       0.43      0.42      0.43       227

     accuracy                           0.39       609
    macro avg       0.39      0.39      0.39       609
 weighted avg       0.39      0.39      0.39       609


Confusion Matrix:
[[71 49 58]
 [65 72 67]
 [70 62 95]]


In [39]:
get_feature_importances(rf_model, X)

,Feature,Importance
0,PageRank,0.251981
3,ArticleRank,0.236873
1,DegreeCentrality,0.232416
2,EigenvectorCentrality,0.229114
5,Louvain,0.025737
4,Leiden,0.023879


In [40]:
get_feature_importances(et_model, X)

,Feature,Importance
0,PageRank,0.246404
1,DegreeCentrality,0.238567
3,ArticleRank,0.238272
2,EigenvectorCentrality,0.237249
4,Leiden,0.019788
5,Louvain,0.019721


## FastRp

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = fast_graph_df.drop(columns=['SpecID', 'Status', 'SurID'])

# Target variable
y = fast_graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.5681444991789819
Extra Trees Accuracy: 0.5681444991789819


Fast rp mixed with traditional graph metrics

In [42]:
joined_graph = pd.merge(graph_df, fast_graph_df, on=['SpecID', 'Status', 'SurID'], how='inner')

In [43]:
joined_graph

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,Status,SurID,embedding_0,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,210407-1-22,1.151713,2301.560127,0.022129,0.238165,2,1122,Hypoglycemia,210407-1,-0.066228,...,-0.093563,0.114487,-0.079192,0.192730,0.296170,-0.097675,0.084518,-0.268402,-0.020220,-0.099258
1,210407-1-29,1.142775,2280.548482,0.021927,0.237365,2,1122,Hypoglycemia,210407-1,-0.066101,...,-0.093855,0.114483,-0.078765,0.193247,0.295801,-0.098212,0.084640,-0.268450,-0.020361,-0.099599
2,210519-1-20,1.141597,2277.600657,0.021898,0.237256,2,1122,Hyperglycemia,210519-1,-0.066402,...,-0.093438,0.114474,-0.079119,0.192910,0.296234,-0.097497,0.084571,-0.268825,-0.020422,-0.099212
3,210511-1-49,1.138794,2270.600562,0.021831,0.236992,2,1122,Hyperglycemia,210511-1,-0.066116,...,-0.093297,0.114333,-0.079095,0.192779,0.296559,-0.097675,0.084300,-0.268270,-0.020429,-0.098753
4,210504-3-36,1.138285,2272.546214,0.021850,0.237036,1593,404,Hypoglycemia,210504-3,-0.065588,...,-0.093982,0.114745,-0.078797,0.192547,0.295602,-0.096752,0.084933,-0.270006,-0.020303,-0.099939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,210303-1-48,0.415566,602.611228,0.005794,0.173155,2,1122,Normal,210303-1,-0.066264,...,-0.098120,0.112739,-0.081020,0.192769,0.303030,-0.100220,0.080012,-0.265384,-0.018892,-0.097580
3041,210324-1-10,0.412717,596.614308,0.005736,0.172919,421,477,Normal,210324-1,-0.065688,...,-0.093793,0.115992,-0.079352,0.182199,0.298915,-0.102581,0.080022,-0.266779,-0.024713,-0.094020
3042,210211-1-08,0.382157,523.342455,0.005032,0.170141,421,477,Hyperglycemia,210211-1,-0.065841,...,-0.096897,0.115750,-0.080682,0.182507,0.303586,-0.102319,0.076999,-0.266769,-0.023462,-0.093129
3043,210303-2-26,0.380200,525.173999,0.005049,0.170157,2,1122,Normal,210303-2,-0.064454,...,-0.091992,0.113707,-0.078143,0.193889,0.297528,-0.096104,0.085794,-0.267116,-0.021879,-0.099715


In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = joined_graph.drop(columns=['SpecID', 'Status', 'SurID'])

# Target variable
y = joined_graph['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.5730706075533661
Extra Trees Accuracy: 0.5796387520525451


In [45]:
evaluate_extra_trees(joined_graph)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3436 +/- 0.0308
